In [1]:
from energyflow.archs import PFN
import numpy as np
import tensorflow as tf
import ray
from ray import tune
from ray.train import Trainer

In [2]:
# pip install ray

In [3]:
# pip install -U tensorboardx

In [4]:
# pip install py-rsync

In [5]:
def compile_model(config):
        compile_opts={ 'loss':'categorical_crossentropy',
                       'optimizer':'adam',
                       'metrics':'acc',
                       'weighted_metrics':["categorical_crossentropy"],
                      }
        print("compile_model...")
        #print("in_dim:{}".format(in_dim))
        Phi_sizes = config['Phi_sizes']
        F_sizes = config['F_sizes']
#         latent_dropout = config['latent_dropout']
#         F_dropouts = config['F_dropouts']
        DSNNr = PFN(input_dim=5,
                    Phi_sizes=Phi_sizes, F_sizes=F_sizes,
                    summary=False,
                    mask_val=-99,
#                     latent_dropout=latent_dropout,
#                     F_dropouts=F_dropouts,
                    compile_opts=compile_opts)
        return DSNNr

In [6]:
checkpoint_dir = tf.keras.callbacks.ModelCheckpoint('./saved_models/' + 'Raytest',
                                 monitor='val_loss',
                                 verbose=2,
                                 save_best_only=True,
                                 mode='min')
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  min_delta=0,
                                  patience=500,
                                  verbose=1,
                                  restore_best_weights=True)
callbacks = [checkpoint_dir, earlyStopping]

def train_and_score(config,checkpoint_dir=None):
        X_train = np.load("/home/jovyan/SplitArraysTest.npz")['X_train']
        Y_train = np.load("/home/jovyan/SplitArraysTest.npz")['Y_train']
        X_test = np.load("/home/jovyan/SplitArraysTest.npz")['X_test']
        Y_test = np.load("/home/jovyan/SplitArraysTest.npz")['Y_test']

        model = compile_model(config)
        print("start to train_and_score the compiling model...")
        model.fit(X_train, Y_train,
                  batch_size=20000,
                  epochs=5,  # using early stopping, so no real limit
                  verbose=0,
                  validation_data=(X_test, Y_test),
                  callbacks=[callbacks])

        score = model.evaluate(X_test, Y_test, verbose=0)
        tune.report(accuracy=score[1])

In [7]:
%%time
analysis = tune.run(
    train_and_score,
    config={
        'Phi_sizes': tune.grid_search([ (120,120,100),(125,125,100)]),
        'F_sizes':tune.grid_search([(120,120,120),(125,125,125)])
#          'latent_dropout': [0.01,0.15, 0.2],
#         'F_dropouts': [0.01,0.15,0.2]
        #"alpha": tune.grid_search([0.001, 0.01, 0.1]),
        #"beta": tune.choice([1, 2, 3])
    },
    resources_per_trial={'gpu': 1})
print("Best config: ", analysis.get_best_config(metric="accuracy", mode="max"))

2021-11-05 23:22:36,774	WARNING services.py:1748 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2021-11-05 23:22:37,747	ERROR syncer.py:75 -- Log sync requires rsync to be installed.


Trial name,status,loc,F_sizes,Phi_sizes
train_and_score_4385f_00000,RUNNING,10.64.1.4:7692,"(120, 120, 120)","(120, 120, 100)"
train_and_score_4385f_00001,PENDING,,"(125, 125, 125)","(120, 120, 100)"
train_and_score_4385f_00002,PENDING,,"(120, 120, 120)","(125, 125, 100)"
train_and_score_4385f_00003,PENDING,,"(125, 125, 125)","(125, 125, 100)"


(ImplicitFunc pid=7692) compile_model...


(pid=7692) 2021-11-05 23:22:40.301966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
(pid=7692) 2021-11-05 23:22:40.310440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
(pid=7692) 2021-11-05 23:22:40.311136: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
(pid=7692) 2021-11-05 23:22:40.312027: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(pid=7692) To enable them in other operations,

(ImplicitFunc pid=7692) start to train_and_score the compiling model...


...

2021-11-05 23:23:43,973	INFO logger.py:706 -- Removed the following hyperparameter values when logging to tensorboard: {'F_sizes': (125, 125, 125), 'Phi_sizes': (125, 125, 100)}


Result for train_and_score_4385f_00003:
  accuracy: 0.9428571462631226
  date: 2021-11-05_23-23-43
  done: false
  experiment_id: 91b9ab67b9d3408ba78675dc6078e446
  hostname: jupyter-fangying
  iterations_since_restore: 1
  node_ip: 10.64.1.4
  pid: 8106
  time_since_restore: 15.564718961715698
  time_this_iter_s: 15.564718961715698
  time_total_s: 15.564718961715698
  timestamp: 1636154623
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 4385f_00003
  
Result for train_and_score_4385f_00003:
  accuracy: 0.9428571462631226
  date: 2021-11-05_23-23-43
  done: true
  experiment_id: 91b9ab67b9d3408ba78675dc6078e446
  experiment_tag: 3_F_sizes=(125, 125, 125),Phi_sizes=(125, 125, 100)
  hostname: jupyter-fangying
  iterations_since_restore: 1
  node_ip: 10.64.1.4
  pid: 8106
  time_since_restore: 15.564718961715698
  time_this_iter_s: 15.564718961715698
  time_total_s: 15.564718961715698
  timestamp: 1636154623
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id

Trial name,status,loc,F_sizes,Phi_sizes,iter,total time (s),accuracy
train_and_score_4385f_00000,TERMINATED,10.64.1.4:7692,"(120, 120, 120)","(120, 120, 100)",1,13.5268,0.10432
train_and_score_4385f_00001,TERMINATED,10.64.1.4:7850,"(125, 125, 125)","(120, 120, 100)",1,13.3425,0.942857
train_and_score_4385f_00002,TERMINATED,10.64.1.4:7978,"(120, 120, 120)","(125, 125, 100)",1,13.8562,0.942857
train_and_score_4385f_00003,TERMINATED,10.64.1.4:8106,"(125, 125, 125)","(125, 125, 100)",1,15.5647,0.942857


2021-11-05 23:23:44,083	INFO tune.py:630 -- Total run time: 66.53 seconds (66.37 seconds for the tuning loop).


Best config:  {'Phi_sizes': (120, 120, 100), 'F_sizes': (125, 125, 125)}
CPU times: user 1.36 s, sys: 413 ms, total: 1.78 s
Wall time: 1min 7s


In [8]:
# pip install tensorboard

In [9]:
%load_ext tensorboard

In [10]:
log_path = "/home/jovyan/ray_results/train_and_score_2021-11-05_23-22-37"
%tensorboard --logdir log_path